In [208]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm

In [209]:
def is_unitary(m):
    return np.allclose(np.eye(len(m)), m.dot(m.T.conj()))

In [318]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            # print(x)
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])
coefficients[:10], bases[:10]
# print(list(zip(coefficients, bases)))

([0.003034656830204855,
  0.003034656830204855,
  0.003034656830204855,
  0.003034656830204855,
  -0.008373361424264817,
  -0.008373361424264817,
  -0.008373361424264817,
  -0.008373361424264817,
  0.00211113766859809,
  0.00211113766859809],
 ['IIIYYIIIYY',
  'IIIXXIIIYY',
  'IIIYYIIIXX',
  'IIIXXIIIXX',
  'YZZZYIIIYY',
  'XZZZXIIIYY',
  'YZZZYIIIXX',
  'XZZZXIIIXX',
  'YZZYIIIIYY',
  'XZZXIIIIYY'])

In [291]:
# comment this out when stuff actually works
coefficients = [0.5, 0.5]
bases = ['XX', 'XX']
numBits = len(bases[0])
hamiltonian = np.zeros((2**numBits, 2**numBits), dtype=np.cdouble)
numTerms = len(bases)
for c, lbl in zip(coefficients[0:numTerms], bases[0:numTerms]):
    op = Operator(Pauli(label=lbl))
    print(op.data)
    hamiltonian += c*op.data

print(hamiltonian)
print(is_unitary(hamiltonian))

# should give you the identity matrix if the hamiltonian is unitary, whose sum is just the length
print(np.sum(hamiltonian.dot(hamiltonian.T.conj()))) #  - np.eye(len(hamiltonian))))
print(np.sum(np.eye(len(hamiltonian))))

[[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]
True
(4+0j)
4.0


In [316]:
t = 1
qc = QuantumCircuit(10)
numTerms = 10
for c, b in zip(coefficients[0:numTerms], bases[0:numTerms]):
    # implement e^{-i*c*b*t}
    q = []
    for i, op in enumerate(b):
        if (op == 'X'):
            # remove redunadant Hadamards
            for gate in qc.data:
                print('\ngate name:', gate[0].name)
                print('qubit(s) acted on:', gate[1][0].index)
                print('other paramters (such as angles):', gate[0].params)

            if (False and len(qc.data) > 0):
                name = qc.data[-1][0].name
                print(qc.data[-1][0].name)
                if (qc.data[-1][0].name == 'h'):
                    print("popped!")
                    qc.data.pop(-1)
            else:
                pass
            
            qc.h(i)
            q.append(i)
        elif (op == 'Z'):
            q.append(i)
        elif (op == 'Y'):
            qc.h(i)
            qc.s(i)
            q.append(i)
        elif (op == 'I'):
            continue
        else:
            print(op, "case not accounted for")
    if (len(q) > 0):
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        qc.rz(2*c*t, q[-1])
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
            elif (op == 'Z'):
                continue
            elif (op == 'Y'):
                qc.sdg(i)
                qc.h(i)
            elif (op == 'I'):
                continue
            else:
                print("case not accounted for")


gate name: h
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: h
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: h
qubit(s) acted on: 1
other paramters (such as angles): []

gate name: cx
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: rz
qubit(s) acted on: 1
other paramters (such as angles): [1.0]

gate name: cx
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: h
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: h
qubit(s) acted on: 1
other paramters (such as angles): []

gate name: h
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: h
qubit(s) acted on: 1
other paramters (such as angles): []

gate name: cx
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: rz
qubit(s) acted on: 1
other paramters (such as angles): [1.0]

gate name: cx
qubit(s) acted on: 0
other paramters (such as angles): []

gate name: h
qubit(s) acted on: 0
other paramters (

/var/folders/ym/0sh6mfr158x1222fb16kkc2w0000gn/T/ipykernel_1477/308418126.py:12: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  print('qubit(s) acted on:', gate[1][0].index)


In [317]:
qc.draw()

┌───┐                   ┌───┐┌───┐                   ┌───┐
q_0: ┤ H ├──■─────────────■──┤ H ├┤ H ├──■─────────────■──┤ H ├
     ├───┤┌─┴─┐┌───────┐┌─┴─┐├───┤├───┤┌─┴─┐┌───────┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ Rz(1) ├┤ X ├┤ H ├┤ H ├┤ X ├┤ Rz(1) ├┤ X ├┤ H ├
     └───┘└───┘└───────┘└───┘└───┘└───┘└───┘└───────┘└───┘└───┘
q_2: ──────────────────────────────────────────────────────────
                                                               
q_3: ──────────────────────────────────────────────────────────
                                                               
q_4: ──────────────────────────────────────────────────────────
                                                               
q_5: ──────────────────────────────────────────────────────────
                                                               
q_6: ──────────────────────────────────────────────────────────
                                                               
q_7: ──────────────────────────────────────────────────────────
                                                               
q_8: ──────────────────────────────────────────────────────────
                                                               
q_9: ──────────────────────────────────────────────────────────

In [203]:
backend = Aer.get_backend('unitary_simulator')
job = execute(qc, backend)
result = job.result()
U_circuit = result.get_unitary(qc, decimals=3) 

In [204]:
diff = U_circuit - expm(-1.j * hamiltonian)
diff

Operator([[-1.00000000e-03-4.99600361e-16j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j, ...,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j],
          [-0.00000000e+00+0.00000000e+00j,
            3.01572472e-01-7.03187341e-01j,
           -2.92577561e-02+3.69567419e-02j, ...,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j],
          [ 0.00000000e+00+0.00000000e+00j,
           -2.92577561e-02+3.69567419e-02j,
            6.69678507e-02-3.52879708e-01j, ...,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j],
          ...,
          [ 0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j, ...,
            3.60636661e-02+2.60122830e

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)

In [205]:
# measure the operator's accuracy
data = diff.data
w, v = LA.eig(data)

# measure the error relative to a matrix of 1s for reference.
ones = np.ones(data.shape)
fakeDiff = U_circuit - ones
w_fake, v_fake = LA.eig(fakeDiff.data)

In [206]:
print(max(w))
print(max(w_fake))

(1.9983506688262593+0.0075052574679956964j)
(0.9990000000000582-0.0339558536926999j)
